# Encapsulin Hits against UniRef90

We've searched our ≈1500 encapsulin hits against UniRef90 - let's see how many of them are truly novel!

In [1]:
import pandas as pd

hits_df = pd.read_csv("../encapsulin_UniRef90_hits.m8", sep="\t", names=["Query", "Target", "Identity", "Alignment Length", "Mismatches", "Gap Openings",
                                                              "Query Start", "Query End", "Target Start", "Target End", "E-Value", "Bitscore"])

hits_df.head()

,Query,Target,Identity,Alignment Length,Mismatches,Gap Openings,Query Start,Query End,Target Start,Target End,E-Value,Bitscore
0,MGYP003108701992,UniRef90_UPI000E6BE03E,0.275,298,205,6,10,302,40,331,4.584000e-17,94
1,MGYP000316064180,UniRef90_A0A7X7EMN5,0.445,301,163,2,7,306,15,312,9.788000e-73,258
2,MGYP003109322860,UniRef90_A0A2D7M0R9,0.492,591,231,15,7,552,2,568,1.786000e-157,516
3,MGYP000384958797,UniRef90_A0A0F9VF95,0.539,404,186,0,4,407,2,405,9.037000e-146,474
4,MGYP003109353800,UniRef90_A0A076GC93,0.295,328,193,12,1,297,1,321,8.156000e-22,109


How many encapsulins have any hits in UniRef90?

In [2]:
len(hits_df["Query"].unique())

1473

That's a lot! But how many have hits with ID above 80%?

In [3]:
len(hits_df[hits_df["Identity"] > 0.8]["Query"].unique())

411

This'll probably be easier to visualize in a graph:

In [20]:
import plotly.express as px

deduplicated_hits = hits_df.sort_values(by="Identity", ascending=False).drop_duplicates(subset="Query")

fig = px.histogram(deduplicated_hits, x="Identity", nbins=10, color_discrete_sequence=["purple"])

fig.update_layout(
    template="plotly_white",
    width=1200,
    height=600,
    title="Sequence Identity of Best Hit in UniRef90"
)

fig.update_traces(marker_line_width=1,marker_line_color="white")

fig.show()